In [ ]:
using Random, LinearAlgebra, Distributions

struct AntSystemParams
    α::Float64
    β::Float64
    Q::Float64
    ρ::Float64
    τ₀::Float64
    num_ants::Int
    max_iterations::Int
end

function generate_distance_matrix(n::Int; seed=123)
    Random.seed!(seed)
    cities = rand(n, 2)
    d = zeros(n, n)
    for i in 1:n
        for j in i+1:n
            dist = norm(cities[i, :] - cities[j, :])
            d[i, j] = dist
            d[j, i] = dist
        end
    end
    return d
end

function construct_path(n::Int, d::Matrix{Float64}, τ::Matrix{Float64}, α::Float64, β::Float64)
    path = Int[]
    visited = falses(n)
    start = rand(1:n)
    push!(path, start)
    visited[start] = true
    current = start
    
    for _ in 2:n
        available = findall(.!visited)
        if isempty(available)
            break
        end
        
        probabilities = zeros(length(available))
        for (idx, city) in enumerate(available)
            η = 1.0 / d[current, city]
            probabilities[idx] = (τ[current, city]^α) * (η^β)
        end
        
        total = sum(probabilities)
        if total ≈ 0.0
            next_city = rand(available)
        else
            probabilities ./= total
            next_city = available[rand(Categorical(probabilities))]
        end
        
        push!(path, next_city)
        visited[next_city] = true
        current = next_city
    end
    return path
end

function calculate_path_length(path::Vector{Int}, d::Matrix{Float64})
    total_length = 0.0
    n = length(path)
    for i in 1:n
        city_i = path[i]
        city_j = (i < n) ? path[i+1] : path[1]
        total_length += d[city_i, city_j]
    end
    return total_length
end

function update_pheromones!(τ::Matrix{Float64}, all_paths::Vector{Vector{Int}}, all_lengths::Vector{Float64}, Q::Float64, ρ::Float64)
    τ .*= (1 - ρ)
    for (path, L) in zip(all_paths, all_lengths)
        if L ≈ 0.0
            continue
        end
        Δτ = Q / L
        m = length(path)
        for i in 1:m
            city_i = path[i]
            city_j = (i < m) ? path[i+1] : path[1]
            τ[city_i, city_j] += Δτ
            τ[city_j, city_i] += Δτ
        end
    end
end

function ant_system(d::Matrix{Float64}; params::AntSystemParams)
    n = size(d, 1)
    τ = fill(params.τ₀, n, n)
    best_path = Int[]
    best_length = Inf
    
    for _ in 1:params.max_iterations
        all_paths = Vector{Vector{Int}}()
        all_lengths = Vector{Float64}()
        
        for _ in 1:params.num_ants
            path = construct_path(n, d, τ, params.α, params.β)
            L = calculate_path_length(path, d)
            push!(all_paths, path)
            push!(all_lengths, L)
            
            if L < best_length
                best_length = L
                best_path = copy(path)
            end
        end
        
        update_pheromones!(τ, all_paths, all_lengths, params.Q, params.ρ)
    end
    
    return best_path, best_length
end

params = AntSystemParams(1.0, 2.0, 100.0, 0.1, 0.1, 10, 100)

n = 10
d = generate_distance_matrix(n)
best_path, best_length = ant_system(d, params=params)

println("Лучший путь: ", best_path)
println("Длина пути: ", best_length)

Лучший путь: [1, 6, 5, 7, 4, 10, 9, 2, 3, 8]
Длина пути: 3.205272051291517
